# Session 3: Spring 2026

Big Data Algorithms

## Outline

Review from the last session:

-   LSH family for minhash signatures

-   AND-OR and OR-AND amplification

-   An example with text data

-   LSH Families for Cosine, Hamming and Euclidean distances

-   Data Streams:

    -   Reservoir Sampling
    -   Counting unique items: the FM Algorithm

## Minhashing: LSH family for Jaccard distance

## AND, OR, AND-OR and OR-AND amplification

## Pairing probabilities

With $n$ signatures arranged in $b$ bands with $r$ rows per band
($n = r \cdot b)$), the probabilities for two documents with similarity
$s$ becoming a **candidate pair** are:

-   **AND-OR**: $1 - (1-s^r)^b$

-   **OR-AND**: $(1- (1-s)^r)^b$

## Exercise (submit your code on Canvas)

Suppose we wish to amplify a $(0.2, 0.6, 0.8, 0.4)$-sensitive family.

How many signatures, rows and bands with what kind (either AND-OR or
OR_AND) of amplification can achieve each of the following goals?

-   Reduce the false positive rate from 0.4 to less than 0.15

-   Reduce the false negative rate from 0.2 to less than 0.1

-   Reduce both rates simultaneously to less than 0.05

## Python Imports

In [ ]:
import math

# Base rates from the (0.2, 0.6, 0.8, 0.4)-sensitive family
p_near = 0.8   # true positive rate (TPR) for similar pairs
p_far  = 0.4   # false positive rate (FPR) for dissimilar pairs

def or_and(p, r, b):
    """
    OR of b bands, each band is AND of r rows:
    p' = 1 - (1 - p^r)^b
    """
    return 1.0 - (1.0 - (p ** r)) ** b

def and_or(p, r, b):
    """
    AND of b groups, each group is OR of r rows:
    p' = (1 - (1 - p)^r)^b
    """
    return (1.0 - (1.0 - p) ** r) ** b

def rates_for_scheme(scheme_fn, r, b):
    """Return amplified (TPR, FPR, FNR) for given scheme and params."""
    tpr = scheme_fn(p_near, r, b)
    fpr = scheme_fn(p_far,  r, b)
    fnr = 1.0 - tpr
    return tpr, fpr, fnr

def find_params(scheme_fn, max_r=40, max_b=60, fpr_max=None, fnr_max=None):
    """
    Find (r,b) meeting constraints. Minimizes total rows used = r*b,
    then breaks ties by smaller b, then smaller r.
    """
    best = None
    best_key = None

    for r in range(1, max_r + 1):
        for b in range(1, max_b + 1):
            tpr, fpr, fnr = rates_for_scheme(scheme_fn, r, b)

            if fpr_max is not None and not (fpr < fpr_max):
                continue
            if fnr_max is not None and not (fnr < fnr_max):
                continue

            key = (r * b, b, r)  # cost metric: total signatures/rows, etc.
            if best is None or key < best_key:
                best = (r, b, tpr, fpr, fnr)
                best_key = key

    return best

def pretty_result(name, res):
    if res is None:
        print(f"{name}: no solution found in search bounds.")
        return
    r, b, tpr, fpr, fnr = res
    print(f"{name}: r={r}, b={b}, total rows r*b={r*b}")
    print(f"  TPR={tpr:.6f}  FPR={fpr:.6f}  FNR={fnr:.6f}")

# --------------------------
# Goal A: FPR < 0.15
# --------------------------
print("GOAL A: Reduce FPR from 0.4 to < 0.15\n")
pretty_result("OR-AND best", find_params(or_and, fpr_max=0.15))
pretty_result("AND-OR best", find_params(and_or, fpr_max=0.15))

# --------------------------
# Goal B: FNR < 0.10
# --------------------------
print("\nGOAL B: Reduce FNR from 0.2 to < 0.10\n")
pretty_result("OR-AND best", find_params(or_and, fnr_max=0.10))
pretty_result("AND-OR best", find_params(and_or, fnr_max=0.10))

# --------------------------
# Goal C: BOTH < 0.05
# --------------------------
print("\nGOAL C: Reduce BOTH FPR and FNR to < 0.05\n")
pretty_result("OR-AND best", find_params(or_and, fpr_max=0.05, fnr_max=0.05))
pretty_result("AND-OR best", find_params(and_or, fpr_max=0.05, fnr_max=0.05))


In [8]:
import numpy as np
import pandas as pd

rng = np.random.default_rng()

In [9]:
import random
import math
import numpy as np
import pandas as pd

rng = np.random.default_rng()
class UHF:
    """A factory for producing a universal family of hash functions"""

    @staticmethod
    def is_prime(k):
        if k%2==0:
            return False
        for i in range(3, int(math.sqrt(k)), 2):
            if k%i == 0:
                return False
        return True

    def __init__(self, n):
        """Universe size is n"""
        self.n = n
        if n%2==0:
            m = n+1
        else:
            m = n+2
        while not(UHF.is_prime(m)):
            m = m+2
        self.p = m

    def make_hash(self, m):
        """Return a random hash function

        m: table size
        """
        a = random.randint(1,self.p-1)
        b = random.randint(0,self.p-1)
        return lambda k: ((a*k+b)%self.p)%m

In [25]:
n = 10000000
uhf = UHF(n)
uhf.p
m=5000
make_hash = uhf.make_hash(m)

In [26]:
p = int(np.random.random()*100000)
q = int(np.random.random()*100000)

In [27]:
hashes = [uhf.make_hash(m) for _ in range(50000)]

In [28]:
sum(i(p) == i(q) for i in hashes)

7

## Test the Universal Hash Family (Exercise)

Set up and evaluate an experiment to verify that the family defined
above is indeed a universal family. Use a reasonable universe of keys,
say integers in `range(1000000)` and a hash table of size $5000$.

-   Fix two keys at random
-   Generate a lot of hash functions and see how many of them collide

## Shingling

To compare **text** data, we create **$k$-shingles** from the data
(sometimes called **$k$-grams**).

> all substrings consisting of $k$ consecutive characters from the text!

Each text document is characterized by its $k$-shingles and their
*frequency*. We can also just use binary indicators for the shingles.

We look for similaries between actual text documents: in this case, a
collection of term papers (extract the zip file `med_doc_set.zip` into a
folder called `med_doc_set`).

## Matrix Representation Of Data

A useful builtin module called `pathlib` can be used to work with files
on the filesystem in Python. For example, let’s collect all the files
are our sub-folder `med_doc_set`.

In [42]:
from pathlib import Path
med_doc_set = Path('med_doc_set/med_doc_set.csv')

In [ ]:
for i in range:1:3

## Scikit-Learn text data preparation

In [35]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

The `CountVectorizer` and `TfidfVectorizer` classes create **models**:
objects that can be used to **fit** data for further analysis. Once
fitted, the resulting model acquires a **term** vocabulary (the
features) that can be **transformed** to a **document-term** matrix, a
starting point for most mining/machine-learning algorithms.

`sklearn` also allows a model to combine the fit and transform steps.

In [41]:
files_iterator = med_doc_set.glob('*.txt')
cv = CountVectorizer(input='filename', encoding='iso-8859-1',
                     decode_error='ignore', analyzer='word',
                     ngram_range=(1,2), stop_words='english',
                     binary=True)
cv_m = cv.fit_transform(files_iterator)

ValueError: empty vocabulary; perhaps the documents only contain stop words

`sklearn` models have attributes whose names end in an underscore: best
to read the documentation! The document-term matrix has terms as columns
and documents as rows.

In [37]:
num_docs, num_features = cv_m.shape
print(num_docs, num_features)

NameError: name 'cv_m' is not defined

## Minhashing with Multiple Signatures & Amplification!

Consider Jaccard similarity for now:

-   Generate 20 different signatures for the documents: signatures
    matrix has signatures as rows and documents as columns

-   Arrange signatures in 5 **bands**, with each band containing 4
    **rows**.

-   **Candidate Pair:** Any pair of documents whose signatures agree in
    **every** row in **some** band.

## Find Candidate Pairs

1.  Hash documents to *buckets* such that similar documents are likely
    to hash to the same bucket!

2.  **Actually compare features** of candidate pairs

Benefits: Instead of $O(n^2)$ comparisons, we only need $O(n)$
comparisons to find *approximately* similar documents!

## General Approach

-   Choose distance measure for items

-   Create a matrix of signatures using an **appropriate LSH family**

-   Construct candidate pairs applying the LSH banding technique

-   Choose a threshold fraction $t$ for *similarity* of items in order
    for them to be regarded as a *true pair*.

-   Check if the signatures for the candidate pairs match in at least a
    fraction $t$, or if pairs are sufficiently similar, do a more
    fine-grained check.

## Cosine Distance

Distances equals angle (between 0 and $\pi$) between points in Euclidean
space

-   Can be computed with dot products of the vectors representing the
    points

-   Distance is a metric

> Note: `scipy` treats the cosine distance as the **cosine** of the
> angle between points. Hence `scipy` cosine “distance’ is not a
> distance metric! However, it is convenient because no `acos` (inverse
> cosine) computations are needed.

## LSH Family for Cosine Distance

-   Data is a collection of $n$-dimensional points (treated as vectors)

-   A “hash function” in the family corresponds to a **random vector**;
    the hash function applied to a point is the **sign of the dot
    product** of the point with the random vector. For angles
    $d_1 < d_2$ in radians, we get a

> $(d_1, d_2, (1-\frac{d_1}{\pi}), (1-\frac{d_2}{\pi}))$-sensitive
> family

**Sketch:** A vector with coordinates $\pm 1$. These are reasonable
approximations to the full LSH family in high dimensions.

## Term Frequency Inverse-Document Frequency

-   Term Frequency ($D_t$): **number** of times a term $t$ occurs in a
    document (`CountVectorizer` does that)

-   Document Frequency ($N_t$): **number** of documents that contain a
    reference to a term $t$

$\textrm{TF-IDF} = D_t * \log \frac{1+N}{1+N_t}$

**TF-IDF** gives more weight to less-frequent terms, and less weight to
more-frequent terms. Here is a `TfidfVectorizer` model for the same
document set.

In [7]:
tfv = TfidfVectorizer(input='filename', encoding='iso-8859-1',
                      decode_error='ignore', analyzer='word',
                      ngram_range=(2,2), stop_words='english')

## Hamming Distance

Distance between 0/1-valued (binary) vectors of length $n$

-   equals the number of bit positions that differ, or can also be
    defined as the *fraction* of bit positions that differ; i.e., it is
    integer-valued between 0 and $n$ (when un-normalized) or $\leq 1$
    (when normalized by $n$).

-   it is a metric distance

## LSH Family for Hamming Distance

-   Data consists of $n$ dimensional **binary** vectors with positions
    indexed from 0 through $(n-1)$.

-   A “hash function” in the family corresponds to a *projection*: the
    projection $h_i$ for $0 \leq i \leq n-1$ extracts the $i^{th}$
    (least significant) bit of a vector.

*Normalized* Hamming distance $\implies$
$(d_1, d_2, (1- \frac{d_1}{n}), (1-\frac{d_2}{n}))$-sensitive family.

## Euclidean Distances

-   $L_1$ distance (also called **Manhattan distance**):

$L_1(p,q) = \sum_{i=0}^{n} |p_i - q_i|$

-   $L_k$ distance: (the $k=2$ case is the familar distance norm)

$L_k(p, q) = \sqrt[k]{\sum_{i=0}^{n-1} |p_i - q_i|^k}$

-   $L_{\infty}$ distance:

$L_{\infty}(p, q) = max_{i} |p_i - q_i|$

## Example: LSH family for Euclidean $L_2$ distance

-   Data is a collection of $2$-dimensional points

-   A “hash function” in the family corresponds to a **random line**
    that is divided into numbered segments (**buckets**) of length $a$

-   The hash application is the bucket number reached when a
    **perpendicular** is dropped from the point onto the line!

> This yields a $(\frac{a}{2}, 2a, \frac{1}{2}, \frac{1}{3})$-sensitive
> hash family.

## Data Structures/Algorithms for Data Streams

Very large volumes of data that requires *immediate* (real-time)
processing or is too large to be *analyzed in time* from archives.

-   Sensor data (e.g. weather information, or sensors used in IOT
    applications)

-   Satellite data

-   Data involved in cyber-security, e.g. IP traffic on a router

-   Stock price fluctuations

-   Identifying frequent search queries

-   Identifying popular products

## Data Summaries and Sampling

We often need:

-   statistical information about *sliding windows*

-   extraction of *representative* or *reliable* samples: subset
    selection

-   filters to select samples

-   approximate counts of distinct elements

In all these applications, **hashing** is the key!

## Reservoir Sampling

We wish to choose a subset (the reservoir) of size $k<<N$ from a data
stream of *unknown* size $N$.

Desired prob. for item to be in reservoir $=k/N$ (i.e., **uniformly
drawn sample**).

Algorithm:

-   Store first $k$ items in the reservoir

-   Item $i>k$ replaces $j^{th}$ **item in reservoir** if $j$ is random
    choice among $[1,i]$ and $j <= k$; otherwise, discard item $i$.

> **Exercise:** Prove that reservoir sampling works!

## Representative subset for ad-hoc queries

Stream consists of tuples with $n$ fields

-   Sampling involves only a few of the fields, the *key* values

-   If we need a subset that is a fraction $p$ of all the keys:

    -   Use a hash function that has $k$ buckets

    -   Hashing should **only** involve the **key values**.

    -   Choose sample if it hashes to the first $p\cdot k$ buckets

    -   Result will have **all** tuples with certain key values.

## Counting Distinct Elements

> Count (approximately) the number of **unique** elements, $m$, in a
> stream.

-   Very large universal set

-   Traditional in-core data structure like a hash table or a balanced
    search tree is impossible due to space or time constraints.

-   But hashing offers a clue: the wider the range (image) of the hash,
    the more likely it is that distinct elements will hash to
    *distinctive values*.

## Experiment

Create a sample of size 1000000 drawn from a geometric distribution with
probability $p=0.15$ of success.

Compute the number of distinct elements.

In [8]:
sample = None
distinct_elements = np.unique(sample)

## Flajolet-Martin Algorithm

Choose a hash function family that maps elements to bit strings of a
certain length (ensure that range is much larger than the domain).

-   Pick **many hash functions** and apply them to every element.

-   **Intuition:**

    -   More distinct items $\implies$ more distinct hash values!

    -   For each hash function, count special bit patterns in the image.

-   Special pattern: long stretches of trailing zero bits

## Estimating $m$

**Tail length:** For element $a$ and hash function $h$, the tail length

$t_{h}(a) = \mbox{number of trailing zeros in }h(a)$.

-   $R_h$: maximum tail length seen with hash function $h$

-   Estimate of $m$ (according to $h$) is $2^{R_h}$.

-   **Note:** Prob. that $t_h(a)$ is *at least* $r$ is $2^{-r}$.

## Exercise

Complete the following definition:

In [9]:
def tail_length(n):
    """Returns the number of trailing zeros in the bit representation

    n (int): number
    """
    pass

Use it with a random hash function $h$ drawn from our universal hash
family to compute an estimate of the number of unique elements of the
geometric random sample.

## Justification

Suppose that $R_h$ is the current (maximum) tail length and $m$ distinct
elements. Is $2^{R_h}$ a good estimate of $m$?

-   Prob. that an element has $<r$ tail length $= (1 -
      2^{-r})$

-   Prob. that all have this property $=(1-2^{-r})^m$, i.e.
    $\approx e^{-m2^{-r}}$.

-   Prob. that at least one has tail length $>=r$ is $p=1 -
      e^{-m2^{-r}}$.

But $p \rightarrow 1$ (resp. $0$) if $m >> 2^{r}$ (resp. $m << 2^{r}$).

## Why Use Many Hashes

-   To even out the estimates, e.g. by averaging the estimates from
    different hash functions

-   Overestimates, in this case, can be problematic: small probability
    for $R_h$ increasing by one but estimate doubles!

-   Solution:

    -   Form large groups of hash functions, with each group large
        enough

    -   Use the medians of the averages within groups.

    -   Space $=$ one tail length per function; elements not stored!

## Exercise

Simulate the Flajolet-Martin algorithm on the random sample above, using
100 hash functions from the universal family to hash the elements.

## Interlude: The Majority Element Problem

You are given an array of $n$ numbers which contains a **majority**
element, i.e. one which has frequency greater than
$\lfloor n/2 \rfloor$. Find that element!

## Algorithm:

The algorithm always maintains a **stored element** and an **associated
counter**.

-   Store the first element and \*\*initialize\* counter to 1
-   For every subsequent element:
    -   if same as stored element, **increment** counter
    -   if different from stored element and counter is zero, replace
        stored element with new element and **initialize** counter to 1
    -   if different from stored element, **decrement** counter

## Counting most frequent elements: Heavy Hitters

Generalizes the majority element problem: one version is the **$k$-heavy
hitters** problem:

> Find the elements that have frequency at least $\frac{n}{k+1}$.

**Observation:** There can be at most $k$ such heavy hitters!

## The Algorithm (Gries-Misra)

Store upto $k$ elements, each with an associated counter. Initially,
nothing is stored and counters are zero.

For every element seen:

-   if it is currently stored, increment its counter by 1
-   if different from all stored elements, and less than $k$ elements
    currently stored, add element with counter initialized to 1.
-   if different from all stored elements, and $k$ items currently
    stored, **decrement** each counter by 1. Remove any element from
    store if its count becomes 0 as a result.

## Analysis

Show that the reported count for each element $t$ in the store is

-   lower bounded by $f_t - \frac{n}{k+1}$
-   upper bounded by $f_t$

where $f_t$ is its actual frequency.

Hence, all heavy hitters will necessarily be stored (but there could be
non-hitters that remain in the store).

## Exercise

Simulate the Gries-Misra algorithm on the random sample above to obtain
the $3$-heavy hitters.

## Estimating Moments

Given a stream over a universal set $S={a_1,\ldots}$ of distinct
elements, let $m_i$ be the number of occurrences of $a_i$.

$k^{th}$ Moment of the stream = $\sum_{i} {m_i}^k$

-   $0^{th}$ moment (assuming the convention that $0^0 =
      0$) is just the number of distinct elements!

-   $1^{st}$ moment is the **total number** of elements in the stream.

-   Problem with higher moments: Space limitation, since we cannot keep
    exact counts of all the frequencies!

-   Let’s see how to estimate second moments.

## Alon-Matias-Szegedy Algorithm

(Temporary) Assumption: Length of stream known to be $n$

**Ingredients**:

-   Uniformly sample a small subset of positions in the stream
    (*variables* $X_i$)

-   Keep track of the frequencies ($f_i$) of the associated elements (at
    sampled positions) from the first sampled occurrence onwards.

Estimated value for second moment based on variable $X_i$ is
$(2\cdot f_i-1)\cdot n$

## Intuition

-   $c_i$ is the frequency *from position $i$ onwards* of element at
    position $i$

-   Summed over all positions $j$ where a particular element occurs, the
    value $\sum_{j} (2\cdot c_j -1)$ is the second moment for that
    element!

-   The multiplicative factor $n$ accounts for the uniform sampling.

## Hand-Written Exercise

Simulate the AMS second-moment sketch on the sequence

\[6, 4, 6, 11, 11, 4, 2, 1, 8, 8, 15, 2, 4, 7, 5, 4\]

by using the positions 1, 5, 7, 8, 11, 12 (assume that positions are
numbered from 1 through 16).

## Generalizations

-   Can handle higher-order moments $X^k$ by estimating $n(v^k
      - (v-1)^k)$ for a small collection of randomly sampled variables

-   What if $n$ is not known in advance?

    -   Just use $n$ when moments need to reported!

    -   Selection of positions can be done using reservoir sampling!